<h1 align='center'> Modelo Lineal de Regresión Simple </h1>

<h3>Autor</h3>

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 

<h3>Fork</h3>

<h3>Referencias</h3>


<h2>1. Introducción </h2>

En este cuaderno se introduce la diferenciación automática usando Tensorflow 2.1 . Como ilustración implementaremos un modelo lineal simple  de regresión.

<h2>2. Importar módulos requeridos </h2>


In [6]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import tensorflow as tf
import random

print(tf.__version__)

2.1.0


<h2> 3. Creación de datos sintéticos  </h2>

Para el ejemplo vamos a asumir que los datos pueden ser modelados mediante una línea recta de la forma

$$
\begin{equation}
y = wx + b 
\end{equation}
$$

en donde $w= 2$ y $b=1$. Vamos a suponer que la variable dependiente $y$ esta afectada por un error de observación aleatorio $e \sim N(0,0.3^2)$. 

Los datos sintéticos son generados como sigue. Generamos $n=120$ valores $x_i$ equidistantes en el intervalo $(0,3)$. Los valores de la variable dependiente $y_i$ son generados usando la ecuación de la recta más el error: $y_i = w x_i +b + e_i$.

In [7]:
w = 1.2
b = 2
n = 1200
scale= 1.0

x = np.linspace(0,6,n)
y = w*x + b + np.random.normal(scale=scale, size= x.shape[0])
# pandas object to descriptive analysis
data = pd.DataFrame({'x': x, 'y': y})

In [8]:
data.head()

,x,y
0,0.000000,2.307600
1,0.005004,2.072407
2,0.010008,0.994219
3,0.015013,1.777229
4,0.020017,1.410617


<h2>4.  Primer acercamiento gráfico a los datos   </h2>

Revisaremos las relaciones entre variables, usando pairplot y un gráfico de correlación

In [ ]:
#sb.scatterplot(data_f, label =" Datos sintéticos")
#sb.pairplot(data_f, diag_kind ="kde")
#plt.show()

# Create an lmplot
grid = sb.lmplot('x', 'y', data, height=7, truncate=True,  markers='.', scatter_kws={"s": 100})

# Rotate the labels on x-axis
grid.set_xticklabels(rotation=30)
# Access the Figure
fig = grid.fig 

# Add a title to the Figure
fig.suptitle('Regresión lineal \n $y = 1.2x + 1 + \epsilon$', fontsize=20)
# Show the plot
plt.show()

In [10]:
correlation_data = data.corr()
correlation_data.style.background_gradient(cmap='coolwarm', axis =None)

,x,y
x,1,0.897203
y,0.897203,1


<h2><span class="header-section-number"> 5 </span> Estadísticas Descriptivas  </h2>

Tendencia central y dispersión

In [11]:
stats = data.describe()
stats = stats.transpose()
stats

,count,mean,std,min,25%,50%,75%,max
x,1200.0,3.000000,1.734218,0.000000,1.500000,3.000000,4.50000,6.000000
y,1200.0,5.591003,2.291917,0.317478,3.739724,5.584183,7.41657,10.917964


<h2>5. Prepara datos para entrenamiento y validación </h2>


In [12]:
# index to sample
size_test = 0.2 # 20% for testing
n_test = np.int(n*size_test )
n_train = np.int(n-n_test)

# index for sample for testing and training
test_id  = np.random.choice(range(x.shape[0]),n_test,replace =False)
train_id = np.setdiff1d(range(x.shape[0]), test_id , assume_unique=True)

# extract the samples
x_test  = x[test_id]
x_train = x[train_id]
y_test  = y[test_id]
y_train = y[train_id]

<h2> 6. Crea una clase Linear Model</h2>

La clase tiene dos métodos: **init** y **call**. *init* inicializa *w* (weight) y *b* (bias) aleatoriamente y *call* retorna los valores usando la ecuación $y = wx + b$.

In [13]:
#LinearModel class
import numpy as np

class LinearModel:
    def __call__(self,x):
        return self.weight * x + self.bias
    
    def __init__(self):
        self.weight = tf.Variable(np.random.rand()) #  generate a value in [0,1)
        self.bias   = tf.Variable(np.random.rand()) #  generate a value in [0,1)


<h2> 7.  Define las funciones pérdida (loss) y entrenamiento (train)</h2>

La función pérdida será el error cuadrático medio definido por

$$
\begin{equation}
loss = \frac{1}{n} \sum_{i=1}^{n} (y_i-\tilde{y}_i)^2,
\end{equation}
$$

en donde $\tilde{y}_i$ es el valor predicho (pred) por el modelo para $x_i$.

En la función de entrenamiento vamos a introducir  diferenciación automática con un contexto de *tf.GradientTape*. El método de optimización es gradiente descendiente,que usa una tasa de aprendizaje *lr*.

In [14]:
# loss function
def loss(y, pred):
    return tf.reduce_mean(tf.math.square(y-pred))

# train function
def train(linear_model, x,y, lr= 0.12):
    with tf.GradientTape() as t:
        t.watch([linear_model.weight, linear_model.bias])
        current_loss = loss(y, linear_model(x))
        
    lr_weight, lr_bias = t.gradient(current_loss, [linear_model.weight, linear_model.bias])
    linear_model.weight.assign_sub(lr*lr_weight) # linear_model.weight is a tensor
    linear_model.bias.assign_sub(lr*lr_bias) # linear_model.bias is a tensor
    

<h2> 8.  Entrenamiento del modelo </h2>

Estamos listos para correr el modelo de regresión lineal. Diremos: *entrenar el modelo*. Definimos un número de epochs (iteraciones). Por defecto usaremos el valor 0.06 como rata de aprendizaje. Usaremos 200 epochs en este experimento.

In [ ]:
linear_model = LinearModel()
weights, biases = [], []
train_loss_p, test_loss_p = [], []
epochs = 200

for epoch in range(epochs):
    weights.append(linear_model.weight.numpy())
    biases.append(linear_model.bias.numpy())
    train(linear_model, x_train, y_train, lr =0.06)
    train_loss = loss(y_train,linear_model(x_train))
    test_loss  = loss(y_test,linear_model(x_test))
    # save loss values to plot
    train_loss_p.append(train_loss)
    test_loss_p.append(test_loss)
    print(f"Epoch {epoch}: train loss {train_loss.numpy()}: test loss {test_loss.numpy()}")

<h2> 9.  Validación. Gráficos de la función de pérdida </h2>

In [ ]:
# plots
def loss_plots(train, test, y_min =0,y_max=2):
    plt.plot(train,color='red', label='train loss')
    plt.plot(test, color='blue', label='test loss')
    plt.ylim(y_min,y_max)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

#
loss_plots(train_loss_p, test_loss_p,y_min=0.5,y_max = 3.0)

<h2> 10.  Extracción de parámetros </h2>

In [17]:
print("weight =", linear_model.weight.numpy())
print("bias =",linear_model.bias.numpy())

weight = 1.193455
bias = 2.0037918
